Добавление объемов не улучшило модель

In [6]:
# # Кодирование объемов по квантилю всей колонки объемов
# import sqlite3
# import torch
# import torch.nn as nn
# import torch.optim as optim
# import numpy as np
# import pandas as pd
# from pathlib import Path
# from torch.utils.data import Dataset, DataLoader

# # === 1. ЗАГРУЗКА ДАННЫХ ===
# db_path = Path(r'C:\Users\Alkor\gd\data_quote_db\RTS_futures_options_day.db')

# with sqlite3.connect(db_path) as conn:
#     df_fut = pd.read_sql_query(
#         "SELECT TRADEDATE, OPEN, LOW, HIGH, CLOSE, VOLUME FROM Futures",
#         conn
#     )

# # === 2. ФУНКЦИЯ КОДИРОВАНИЯ СВЕЧЕЙ (ЛИХОВИДОВ) С УЧЕТОМ ОБЪЕМА ===
# low_quantile = df_fut['VOLUME'].quantile(0.33)
# high_quantile = df_fut['VOLUME'].quantile(0.66)

# def encode_volume(volume):
#     if volume <= low_quantile:
#         return '0'  # Низкий объем
#     elif volume <= high_quantile:
#         return '1'  # Средний объем
#     else:
#         return '2'  # Высокий объем

# def encode_candle(row):
#     open_, low, high, close, volume = row['OPEN'], row['LOW'], row['HIGH'], row['CLOSE'], row['VOLUME']

#     if close > open_:
#         direction = 1  
#     elif close < open_:
#         direction = 0  
#     else:
#         direction = 2  

#     upper_shadow = high - max(open_, close)
#     lower_shadow = min(open_, close) - low
#     body = abs(close - open_)

#     def classify_shadow(shadow, body):
#         if shadow < 0.1 * body:
#             return 0  
#         elif shadow < 0.5 * body:
#             return 1  
#         else:
#             return 2  

#     upper_code = classify_shadow(upper_shadow, body)
#     lower_code = classify_shadow(lower_shadow, body)

#     volume_code = encode_volume(volume)  # Код объема

#     return f"{direction}{upper_code}{lower_code}{volume_code}"  # Код свечи + объем

# df_fut['CANDLE_CODE'] = df_fut.apply(encode_candle, axis=1)

# # === 3. ПОДГОТОВКА ДАННЫХ ===
# unique_codes = sorted(df_fut['CANDLE_CODE'].unique())  # Учитывает новые коды с объемом
# code_to_int = {code: i for i, code in enumerate(unique_codes)}
# df_fut['CANDLE_INT'] = df_fut['CANDLE_CODE'].map(code_to_int)

# window_size = 20  
# predict_offset = 1  

# X, y = [], []
# for i in range(len(df_fut) - window_size - predict_offset):
#     X.append(df_fut['CANDLE_INT'].iloc[i:i+window_size].values)
    
#     # Новый таргет: 1 — рост, 0 — падение
#     y.append(1 if df_fut['CLOSE'].iloc[i+window_size+predict_offset] > df_fut['CLOSE'].iloc[i+window_size] else 0)

# X, y = np.array(X), np.array(y)

# split = int(0.8 * len(X))
# X_train, y_train = X[:split], y[:split]
# X_test, y_test = X[split:], y[split:]

# class CandlestickDataset(Dataset):
#     def __init__(self, X, y):
#         self.X = torch.tensor(X, dtype=torch.long)
#         self.y = torch.tensor(y, dtype=torch.float32)

#     def __len__(self):
#         return len(self.X)

#     def __getitem__(self, idx):
#         return self.X[idx], self.y[idx]

# train_dataset = CandlestickDataset(X_train, y_train)
# test_dataset = CandlestickDataset(X_test, y_test)

# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# # === 4. СОЗДАНИЕ НЕЙРОСЕТИ (LSTM) ===
# class CandleLSTM(nn.Module):
#     def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
#         super(CandleLSTM, self).__init__()
#         self.embedding = nn.Embedding(vocab_size, embedding_dim)
#         self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
#         self.fc = nn.Linear(hidden_dim, output_dim)
#         self.sigmoid = nn.Sigmoid()

#     def forward(self, x):
#         x = self.embedding(x)
#         x, _ = self.lstm(x)
#         x = self.fc(x[:, -1, :])  
#         return self.sigmoid(x)

# # === 5. ОБУЧЕНИЕ МОДЕЛИ С СОХРАНЕНИЕМ ЛУЧШЕЙ ===
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# model = CandleLSTM(vocab_size=len(unique_codes), embedding_dim=8, hidden_dim=32, output_dim=1).to(device)
# criterion = nn.BCELoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)

# best_accuracy = 0  
# model_path = "best_model_05.pth"

# epochs = 2000
# for epoch in range(epochs):
#     model.train()
#     total_loss = 0
#     for X_batch, y_batch in train_loader:
#         X_batch, y_batch = X_batch.to(device), y_batch.to(device)

#         optimizer.zero_grad()
#         y_pred = model(X_batch).squeeze()
#         loss = criterion(y_pred, y_batch)
#         loss.backward()
#         optimizer.step()

#         total_loss += loss.item()

#     # === Проверка на тесте после каждой эпохи ===
#     model.eval()
#     correct = 0
#     total = 0
#     with torch.no_grad():
#         for X_batch, y_batch in test_loader:
#             X_batch, y_batch = X_batch.to(device), y_batch.to(device)
#             y_pred = model(X_batch).squeeze().round()
#             correct += (y_pred == y_batch).sum().item()
#             total += y_batch.size(0)

#     accuracy = correct / total
#     print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader):.4f}, Test Accuracy: {accuracy:.2%}")

#     # === Сохранение лучшей модели ===
#     if accuracy > best_accuracy:
#         best_accuracy = accuracy
#         torch.save(model.state_dict(), model_path)
#         print(f"✅ Model saved with accuracy: {best_accuracy:.2%}")

# # === 6. ЗАГРУЗКА ЛУЧШЕЙ МОДЕЛИ И ТЕСТ ===
# print("\n🔹 Loading best model for final evaluation...")
# model.load_state_dict(torch.load(model_path))
# model.eval()

# correct = 0
# total = 0
# with torch.no_grad():
#     for X_batch, y_batch in test_loader:
#         X_batch, y_batch = X_batch.to(device), y_batch.to(device)
#         y_pred = model(X_batch).squeeze().round()
#         correct += (y_pred == y_batch).sum().item()
#         total += y_batch.size(0)

# final_accuracy = correct / total
# print(f"🏆 Final Test Accuracy: {final_accuracy:.2%}")


In [7]:
# # Кодирование объемов (2 КЛАССА: УМЕНЬШИЛСЯ / УВЕЛИЧИЛСЯ ИЛИ НЕ ИЗМЕНИЛСЯ)
# import sqlite3
# import torch
# import torch.nn as nn
# import torch.optim as optim
# import numpy as np
# import pandas as pd
# from pathlib import Path
# from torch.utils.data import Dataset, DataLoader

# # === 1. ЗАГРУЗКА ДАННЫХ ===
# db_path = Path(r'C:\Users\Alkor\gd\data_quote_db\RTS_futures_options_day.db')

# with sqlite3.connect(db_path) as conn:
#     df_fut = pd.read_sql_query(
#         "SELECT TRADEDATE, OPEN, LOW, HIGH, CLOSE, VOLUME FROM Futures",
#         conn
#     )

# # === 2. ФУНКЦИЯ КОДИРОВАНИЯ СВЕЧЕЙ (ЛИХОВИДОВ) ===
# def encode_candle(row):
#     open_, low, high, close = row['OPEN'], row['LOW'], row['HIGH'], row['CLOSE']

#     if close > open_:
#         direction = 1  
#     elif close < open_:
#         direction = 0  
#     else:
#         direction = 2  

#     upper_shadow = high - max(open_, close)
#     lower_shadow = min(open_, close) - low
#     body = abs(close - open_)

#     def classify_shadow(shadow, body):
#         if shadow < 0.1 * body:
#             return 0  
#         elif shadow < 0.5 * body:
#             return 1  
#         else:
#             return 2  

#     upper_code = classify_shadow(upper_shadow, body)
#     lower_code = classify_shadow(lower_shadow, body)

#     return f"{direction}{upper_code}{lower_code}"

# df_fut['CANDLE_CODE'] = df_fut.apply(encode_candle, axis=1)

# # === 3. КОДИРОВАНИЕ ОБЪЕМОВ (2 КЛАССА: УМЕНЬШИЛСЯ / УВЕЛИЧИЛСЯ ИЛИ НЕ ИЗМЕНИЛСЯ) ===
# df_fut['VOLUME_CHANGE'] = 1  # По умолчанию (для первой строки)
# df_fut.loc[1:, 'VOLUME_CHANGE'] = np.where(
#     df_fut['VOLUME'].iloc[1:].values < df_fut['VOLUME'].shift(1).iloc[1:].values, 0, 1
# )

# # Объединяем код свечи и код объема в один код
# df_fut['COMBINED_CODE'] = df_fut['CANDLE_CODE'] + df_fut['VOLUME_CHANGE'].astype(str)

# # === 4. ПОДГОТОВКА ДАННЫХ ДЛЯ НЕЙРОСЕТИ ===
# unique_codes = sorted(df_fut['COMBINED_CODE'].unique())
# code_to_int = {code: i for i, code in enumerate(unique_codes)}
# df_fut['CANDLE_INT'] = df_fut['COMBINED_CODE'].map(code_to_int)

# window_size = 20  
# predict_offset = 1  

# X, y = [], []
# for i in range(len(df_fut) - window_size - predict_offset):
#     X.append(df_fut['CANDLE_INT'].iloc[i:i+window_size].values)
#     y.append(1 if df_fut['CLOSE'].iloc[i+window_size+predict_offset] > df_fut['CLOSE'].iloc[i+window_size] else 0)

# X, y = np.array(X), np.array(y)

# split = int(0.8 * len(X))
# X_train, y_train = X[:split], y[:split]
# X_test, y_test = X[split:], y[split:]

# class CandlestickDataset(Dataset):
#     def __init__(self, X, y):
#         self.X = torch.tensor(X, dtype=torch.long)
#         self.y = torch.tensor(y, dtype=torch.float32)

#     def __len__(self):
#         return len(self.X)

#     def __getitem__(self, idx):
#         return self.X[idx], self.y[idx]

# train_dataset = CandlestickDataset(X_train, y_train)
# test_dataset = CandlestickDataset(X_test, y_test)

# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# # === 5. СОЗДАНИЕ НЕЙРОСЕТИ (LSTM) ===
# class CandleLSTM(nn.Module):
#     def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
#         super(CandleLSTM, self).__init__()
#         self.embedding = nn.Embedding(vocab_size, embedding_dim)
#         self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
#         self.fc = nn.Linear(hidden_dim, output_dim)
#         self.sigmoid = nn.Sigmoid()

#     def forward(self, x):
#         x = self.embedding(x)
#         x, _ = self.lstm(x)
#         x = self.fc(x[:, -1, :])  
#         return self.sigmoid(x)

# # === 6. ОБУЧЕНИЕ МОДЕЛИ С СОХРАНЕНИЕМ ЛУЧШЕЙ ===
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# model = CandleLSTM(vocab_size=len(unique_codes), embedding_dim=8, hidden_dim=32, output_dim=1).to(device)
# criterion = nn.BCELoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)

# best_accuracy = 0  
# model_path = "best_model_05.pth"

# epochs = 2000
# for epoch in range(epochs):
#     model.train()
#     total_loss = 0
#     for X_batch, y_batch in train_loader:
#         X_batch, y_batch = X_batch.to(device), y_batch.to(device)

#         optimizer.zero_grad()
#         y_pred = model(X_batch).squeeze()
#         loss = criterion(y_pred, y_batch)
#         loss.backward()
#         optimizer.step()

#         total_loss += loss.item()

#     # === Проверка на тесте после каждой эпохи ===
#     model.eval()
#     correct = 0
#     total = 0
#     with torch.no_grad():
#         for X_batch, y_batch in test_loader:
#             X_batch, y_batch = X_batch.to(device), y_batch.to(device)
#             y_pred = model(X_batch).squeeze().round()
#             correct += (y_pred == y_batch).sum().item()
#             total += y_batch.size(0)

#     accuracy = correct / total
#     print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader):.4f}, Test Accuracy: {accuracy:.2%}")

#     # === Сохранение лучшей модели ===
#     if accuracy > best_accuracy:
#         best_accuracy = accuracy
#         torch.save(model.state_dict(), model_path)
#         print(f"✅ Model saved with accuracy: {best_accuracy:.2%}")

# # === 7. ЗАГРУЗКА ЛУЧШЕЙ МОДЕЛИ И ТЕСТ ===
# print("\n🔹 Loading best model for final evaluation...")
# model.load_state_dict(torch.load(model_path))
# model.eval()

# correct = 0
# total = 0
# with torch.no_grad():
#     for X_batch, y_batch in test_loader:
#         X_batch, y_batch = X_batch.to(device), y_batch.to(device)
#         y_pred = model(X_batch).squeeze().round()
#         correct += (y_pred == y_batch).sum().item()
#         total += y_batch.size(0)

# final_accuracy = correct / total
# print(f"🏆 Final Test Accuracy: {final_accuracy:.2%}")


In [8]:
# Квантиль объема теперь рассчитывается на основе 10 предыдущих свечей, а не на всей выборке
import sqlite3
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import random
from pathlib import Path
from torch.utils.data import Dataset, DataLoader

# === 1. ФИКСАЦИЯ СЛУЧАЙНЫХ ЧИСЕЛ ДЛЯ ДЕТЕРМИНИРОВАННОСТИ ===
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


# 39, 50, 85, 90
set_seed(90)  # Устанавливаем одинаковый seed

# === 1. ЗАГРУЗКА ДАННЫХ ===
db_path = Path(r'C:\Users\Alkor\gd\data_quote_db\RTS_futures_options_day.db')

with sqlite3.connect(db_path) as conn:
    df_fut = pd.read_sql_query(
        "SELECT TRADEDATE, OPEN, LOW, HIGH, CLOSE, VOLUME FROM Futures",
        conn
    )

# === 2. ФУНКЦИЯ КОДИРОВАНИЯ СВЕЧЕЙ (ЛИХОВИДОВ) С ДИНАМИЧЕСКИМ КОДИРОВАНИЕМ ОБЪЕМА ===
window_size_volume = 10  # Количество свечей для расчета квантилей

def encode_volume(index, volume):
    if index < window_size_volume:  
        return '1'  # Для первых 10 свечей устанавливаем средний объем

    past_volumes = df_fut['VOLUME'].iloc[index - window_size_volume:index]  
    low_quantile = past_volumes.quantile(0.33)
    high_quantile = past_volumes.quantile(0.66)

    if volume <= low_quantile:
        return '0'  # Низкий объем
    elif volume <= high_quantile:
        return '1'  # Средний объем
    else:
        return '2'  # Высокий объем

def encode_candle(index, row):
    open_, low, high, close, volume = row['OPEN'], row['LOW'], row['HIGH'], row['CLOSE'], row['VOLUME']

    if close > open_:
        direction = 1  
    elif close < open_:
        direction = 0  
    else:
        direction = 2  

    upper_shadow = high - max(open_, close)
    lower_shadow = min(open_, close) - low
    body = abs(close - open_)

    def classify_shadow(shadow, body):
        if shadow < 0.1 * body:
            return 0  
        elif shadow < 0.5 * body:
            return 1  
        else:
            return 2  

    upper_code = classify_shadow(upper_shadow, body)
    lower_code = classify_shadow(lower_shadow, body)

    volume_code = encode_volume(index, volume)  # Код объема

    return f"{direction}{upper_code}{lower_code}{volume_code}"  # Код свечи + объем

df_fut['CANDLE_CODE'] = [encode_candle(i, row) for i, row in df_fut.iterrows()]

# === 3. ПОДГОТОВКА ДАННЫХ ===
unique_codes = sorted(df_fut['CANDLE_CODE'].unique())  
code_to_int = {code: i for i, code in enumerate(unique_codes)}
df_fut['CANDLE_INT'] = df_fut['CANDLE_CODE'].map(code_to_int)

window_size = 20  
predict_offset = 1  

X, y = [], []
for i in range(len(df_fut) - window_size - predict_offset):
    # Add features
    X.append(df_fut['CANDLE_INT'].iloc[i:i+window_size].values)
    
    # Новый таргет: 1 — рост, 0 — падение
    y.append(
        1 if df_fut['CLOSE'].iloc[i+window_size+predict_offset] > 
        df_fut['CLOSE'].iloc[i+window_size] else 0
        )

X, y = np.array(X), np.array(y)

split = int(0.8 * len(X))
X_train, y_train = X[:split], y[:split]
X_test, y_test = X[split:], y[split:]

class CandlestickDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.long)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

def seed_worker(worker_id):
    np.random.seed(42 + worker_id)
    random.seed(42 + worker_id)

train_dataset = CandlestickDataset(X_train, y_train)
test_dataset = CandlestickDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, worker_init_fn=seed_worker)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, worker_init_fn=seed_worker)

# === 4. СОЗДАНИЕ НЕЙРОСЕТИ (LSTM) ===
class CandleLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(CandleLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.lstm(x)
        x = self.fc(x[:, -1, :])  
        return self.sigmoid(x)

# === 5. ОБУЧЕНИЕ МОДЕЛИ С СОХРАНЕНИЕМ ЛУЧШЕЙ ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = CandleLSTM(
    vocab_size=len(unique_codes), embedding_dim=8, hidden_dim=32, output_dim=1
    ).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

best_accuracy = 0  
epoch_best_accuracy = 0
model_path = "bm_fut_lih_vol_02.pth"
early_stop_epochs = 200
epochs_no_improve = 0

epochs = 2000
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        y_pred = model(X_batch).squeeze()
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # === Проверка на тесте после каждой эпохи ===
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            y_pred = model(X_batch).squeeze().round()
            correct += (y_pred == y_batch).sum().item()
            total += y_batch.size(0)

    accuracy = correct / total
    print(
        f"Epoch {epoch+1}/{epochs}, "
        f"Loss: {total_loss/len(train_loader):.4f}, "
        f"Test Accuracy: {accuracy:.2%}, "
        f"Best accuracy: {best_accuracy:.2%}, "
        f"Epoch best accuracy: {epoch_best_accuracy}, "
        f"seed: 90"
        )

    # === Сохранение лучшей модели ===
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        epochs_no_improve = 0
        epoch_best_accuracy = epoch + 1
        torch.save(model.state_dict(), model_path)
        print(f"✅ Model saved with accuracy: {best_accuracy:.2%}")
    else:
        epochs_no_improve += 1

    # === Ранняя остановка ===
    if epochs_no_improve >= early_stop_epochs:
        print(f"🛑 Early stopping at epoch {epoch + 1}")
        break

# === 6. ЗАГРУЗКА ЛУЧШЕЙ МОДЕЛИ И ТЕСТ ===
print("\n🔹 Loading best model for final evaluation...")
model.load_state_dict(torch.load(model_path))
model.eval()

correct = 0
total = 0
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        y_pred = model(X_batch).squeeze().round()
        correct += (y_pred == y_batch).sum().item()
        total += y_batch.size(0)

final_accuracy = correct / total
print(f"🏆 Final Test Accuracy: {final_accuracy:.2%}")

Epoch 1/2000, Loss: 0.6920, Test Accuracy: 50.50%, Best accuracy: 0.00%, Epoch best accuracy: 0, seed: 90
✅ Model saved with accuracy: 50.50%
Epoch 2/2000, Loss: 0.6910, Test Accuracy: 50.50%, Best accuracy: 50.50%, Epoch best accuracy: 1, seed: 90
Epoch 3/2000, Loss: 0.6909, Test Accuracy: 50.89%, Best accuracy: 50.50%, Epoch best accuracy: 1, seed: 90
✅ Model saved with accuracy: 50.89%
Epoch 4/2000, Loss: 0.6897, Test Accuracy: 51.88%, Best accuracy: 50.89%, Epoch best accuracy: 3, seed: 90
✅ Model saved with accuracy: 51.88%
Epoch 5/2000, Loss: 0.6886, Test Accuracy: 51.29%, Best accuracy: 51.88%, Epoch best accuracy: 4, seed: 90
Epoch 6/2000, Loss: 0.6874, Test Accuracy: 51.29%, Best accuracy: 51.88%, Epoch best accuracy: 4, seed: 90
Epoch 7/2000, Loss: 0.6855, Test Accuracy: 49.70%, Best accuracy: 51.88%, Epoch best accuracy: 4, seed: 90
Epoch 8/2000, Loss: 0.6829, Test Accuracy: 51.29%, Best accuracy: 51.88%, Epoch best accuracy: 4, seed: 90
Epoch 9/2000, Loss: 0.6801, Test Accu

In [9]:
# Загружаем модель
model.load_state_dict(torch.load("bm_fut_lih_vol_02.pth"))
model.eval()

# Берем последние 20 свечей из df_fut
last_sequence = torch.tensor(
    df_fut['CANDLE_INT'].iloc[-20:].values, dtype=torch.long
    ).unsqueeze(0).to(device)

# Предсказание
with torch.no_grad():
    probability_up = model(last_sequence).item()  # Вероятность роста
    probability_down = 1 - probability_up  # Вероятность падения

    direction = "📈 UP" if probability_up >= 0.5 else "📉 DOWN"

    print(f"Prediction: {direction}")
    print(
        f"UP Probability: {probability_up:.2%}, \n"
        f"DOWN Probability: {probability_down:.2%}"
        )

Prediction: 📉 DOWN
UP Probability: 48.32%, 
DOWN Probability: 51.68%


In [10]:
df_fut

,TRADEDATE,OPEN,LOW,HIGH,CLOSE,VOLUME,CANDLE_CODE,CANDLE_INT
0,2015-01-05,76930.0,72470.0,78980.0,74600.0,372848,0221,25
1,2015-01-06,74470.0,71200.0,74610.0,73480.0,319307,0121,16
2,2015-01-08,73490.0,71000.0,81380.0,79980.0,537469,1111,40
3,2015-01-09,79950.0,74450.0,81050.0,77650.0,592715,0121,16
4,2015-01-12,77210.0,73180.0,77550.0,73900.0,440908,0111,13
...,...,...,...,...,...,...,...,...
2537,2025-02-12,98600.0,98120.0,101480.0,100700.0,124069,1112,41
2538,2025-02-13,100570.0,100080.0,111010.0,109930.0,247083,1102,38
2539,2025-02-14,109750.0,106470.0,114130.0,108620.0,282242,0222,26
2540,2025-02-17,108960.0,108050.0,115370.0,115170.0,183356,1012,32
